In [22]:
{str(ii):float(vv) for ii,vv in enumerate(range(11, 20))}

{'0': 11.0,
 '1': 12.0,
 '2': 13.0,
 '3': 14.0,
 '4': 15.0,
 '5': 16.0,
 '6': 17.0,
 '7': 18.0,
 '8': 19.0}

In [3]:
[ [c,c+1] for c in range(1,10)]

[[1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10]]

In [8]:
[ c for c in range(1,10) if c % 2 == 0]

[2, 4, 6, 8]

In [23]:
[ lambda c: c+1 for c in range(1,10) if c % 3 == 0]

[<function __main__.<listcomp>.<lambda>(c)>,
 <function __main__.<listcomp>.<lambda>(c)>,
 <function __main__.<listcomp>.<lambda>(c)>]

In [13]:
[ c:c for c in range(1,10) if c % 3 == 0]

SyntaxError: invalid syntax (<ipython-input-13-25ee24e2ddb0>, line 1)

In [26]:
(5 % 2 and 'odd' or 'even')

'odd'

In [27]:
ii = 10

In [28]:
type(ii)

int

In [29]:
ii = 10.0

In [30]:
type(ii)

float

In [2]:
x = lambda arg: print(arg)
x("help")

help
